In [2]:
from py2neo import Graph, Node, Relationship
from pandas import DataFrame
#import json
#json.dumps(list)

In [3]:
graph = Graph("neo4j+s://44bb2475.databases.neo4j.io", auth=("neo4j", "n04yHsQNfrl_f72g79zqMO8xVU2UvUsNJsafcZMtCFM"))

In [55]:
dtf_data = DataFrame([dict(_) for _ in graph.run("MATCH (u:user) RETURN DISTINCT u.fname, u.lname, u.pos, u.email")])
dtf_data

,u.fname,u.lname,u.pos,u.email
0,Liz,Holman,MLex,eaholman@lbl.gov
1,Howard,Yanxon,MLex,yanxonh@anl.gov
2,Hari,Krish,admin,hk@yahoo.com


In [4]:
### SERVICE_API ###
def add_user(fname:str, lname:str, email:str):
    dbindx = len([dict(_) for _ in graph.run("MATCH (u:user) RETURN u")]) + 1
    temp_id = str(fname[0] + lname + str(dbindx).zfill(5))
    cypher_query = '''
    CREATE(n:user {uid:$temp_id, fname:$fname, lname:$lname, email:$email})
    '''
    graph.run(cypher_query, parameters = {'temp_id':temp_id, 'fname':fname, 'lname':lname, 'email':email})

def add_asset(aname:str, atype:str, a_uid:str, owner:str):
    cypher_query = '''
    MATCH (u:user {uid:$owner})
    CREATE (u)-[r:own_asset]->(n:asset {aname:$aname, atype:$atype, a_uid:$a_uid})
    '''
    graph.run(cypher_query, parameters = {'owner':owner, 'aname':aname, 'atype':atype, 'a_uid':a_uid})

def share_asset_read(aname:str, target_uid:str):
    # gives read-only privileges
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (a:asset {aname:$aname})
    CREATE (u) -[r:can_read]-> (a)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def share_asset_edit(aname:str, target_uid:str):
    # gives read/write privileges
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (a:asset {aname:$aname})
    CREATE (u) -[r:can_edit]-> (a)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def assign_asset_manager(aname:str, target_uid:str):
    # gives read/write/share privileges
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (a:asset {aname:$aname})
    CREATE (u) -[r:can_manage]-> (a)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def delete_asset(aname:str):
    # deletes an asset node and all of its relationships
    cypher_query = '''
    MATCH (a:asset {aname:$aname})
    DETACH DELETE a
    '''
    graph.run(cypher_query, parameters = {'aname':aname})

#def remove_asset_access(aname:str, target_uid:str):

### FETCH INFORMATION ###
def get_user_info(uid:str):
    cypher_query = '''
    MATCH (u:user) WHERE u.uid = $uid
    RETURN DISTINCT u.fname as Firstname, u.lname as Lastname, u.email as Email
    '''
    output = DataFrame([dict(_) for _ in graph.run(cypher_query, parameters = {'uid':uid})])
    return output

### MANAGE ROLE NODES ###
def assign_role(uid:str, role_id:str):
    # assign a user by uid to a role by role_id
    cypher_query = '''
    MATCH (u:user {uid:$uid}), (id:role {role_id:$role_id})
    CREATE(u)-[r:has_role]->(id)
    '''
    graph.run(cypher_query, parameters = {'uid':uid, 'role_id':role_id})

def create_role(rname:str, role_id:str):
    # create a new role
    cypher_query = '''
    CREATE(n:role {rname:$rname, role_id:$role_id})
    '''
    graph.run(cypher_query, parameters = {'rname':rname, 'role_id':role_id})


In [ ]:
#def request_asset_access():

#def request_compute_resources():

#def delete_user(uid:str):

# def add_user_list(label, list):
#     # takes a list of user nodes and creates a label and applies label to user nodes
#     cypher_query = '''
#     '''

In [4]:
# create three new roles
# create_role('admin', 'admin')
# create_role('subadmin', 'mlex')
# create_role('general user', 'gen_user')

# create four new users and assign their roles
add_user("Hari","Krish", "hk@lbl.gov")
#assign_role('HKrish00001','admin')

add_user("John", "Doe", "jdoe@yahoo.com")
#assign_role('JDoe00002', 'gen_user')

add_user("Howard", "Yanxon", "yanxonh@anl.gov")
#assign_role('HYanxon00003', 'mlex')

add_user("Alex", "Hexemer", "ahexemer@lbl.gov")
#assign_role('AHexemer00004', 'mlex')

In [12]:
# add one file asset
add_asset("testini", "file", "testini9324", "JDoe000002")

In [166]:
# share file asset in three different ways
share_asset_edit("testini", "HYanxon00003")
share_asset_read("testini", "AHexemer00004")
assign_asset_manager("testini", "HKrish00001")

In [158]:
get_user_info("AHexemer00004")

,Firstname,Lastname,Email
0,Alex,Hexemer,ahexemer@lbl.gov
